In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from keras import layers
from keras.models import Sequential 
from keras.layers import LSTM , Dense 
from keras.callbacks import TensorBoard
import os 
import gtts 
from playsound import playsound 
import cv2 
import time 
import mediapipe as mp 
import numpy as np 
import gtts 
from playsound import playsound 
import shutil

#------------------------------------------------------------------------
import json 
import os 
import cv2
import mediapipe as mp 
import numpy as np 
import threading 
import numpy as np
import time 

In [ ]:
def mediapipe_detection(image, model):
    """
    Performs detection using the MediaPipe library.

    Args:
        image (numpy.ndarray)  : The input image to be processed. It should be in BGR format.
        model (MediaPipe Model): The detection model to be used for processing the image.

    Returns:
        image (numpy.ndarray)      : The processed image with detections overlaid. It is in BGR format.
        results (MediaPipe Results): The detection results obtained from the model.
    """
    # Convert image to RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Make the image writeable flag as False to improve performance
    image.flags.writeable = False

    # Perform detection using the provided model
    results = model.process(image)

    # Make the image writeable flag as True to modify it
    image.flags.writeable = True

    # Convert the image back to BGR format
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Return the processed image and the detection results
    return image, results


In [ ]:

temp_nose_x = None ; 
temp_nose_y = None ; 


def extract_keypoints(results,temp_nose_x,temp_nose_y):
    """
    Extracting the Holistic keypoints from the left and the right hand. 
    Args:
    results ()        : the detection results obtained from the image 
    temp_nose_x (int) : the X coordinate of the clown point 
    temp_nose_y (int) : the Y coordinate of the clown point 
       

    Returns:
    lh (numpy.ndarray) :  the coordinates of the left  hand keypoints in the image
    rh (numpy.ndarray) :  the coordinates of the right hand keypoints in the image
      

    Description:
    
    This function holds significant importance in the project as it is responsible for 
    extracting holistic features from both hands and formatting them appropriately for 
    training purposes.

    One impressive aspect of this function is its ability to make the model translation-invariant.
    This means that regardless of whether the user is sitting with respect to the camera, 
    the model remains capable of accurately detecting the sign.

    The clown point, extracted from the pose model in mediapipe, plays a crucial role in this function. 
    The clown point is located at the edge of the nose and serves as a reference point for all other 
    points. By utilizing this point, the model learns the distance between the holistic features in 
    the hand and the clown point, effectively enhancing its translation invariance.
    """
    try:
        # getting the x coordinate of the clown point 
        # if it does not exist use the previous one (some times the model will not detect the nose )
        # so we will use the previous one 
        X_Nose=results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x
        temp_nose_x=X_Nose
    except : 
        X_Nose = temp_nose_x
    try : 
        Y_Nose=results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y
        temp_nose_y=Y_Nose
    except : 
        Y_Nose= temp_nose_y 
    lh = np.array([[result.x - X_Nose, result.y - Y_Nose, result.z] for result in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[result.x - X_Nose, result.y - Y_Nose, result.z] for result in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3) 
    return np.concatenate([lh,rh])

In [ ]:
# loading the holistic model 
mp_holistic = mp.solutions.holistic
# loading the pose model 
mp_pose = mp.solutions.pose


In [ ]:
# determing the number of frames that will be extracted from one video 

frames_num=40 ; 
def generate_training_data(all_vids,path_to_working_dir):
    """
    Generating Training Data

    Args:
    all_Vids (list)          : A list of all sign folders, e.g., ["Hello", "Live", "Love"].
    path_to_working_dir (str): The path to the directory of the videos.
    
    
    Returns:
    None: This function does not return anything; it saves the keypoints in the file system.
    
    
    Description:
    This function takes the path to the sign folders and iterates over each sign in the folder.
    It extracts the features for each video in the sign folder, concatenates them, and saves them.
    """
    # loop over all the signs 
    for j,k in enumerate(all_vids):
        # enter every sign folder 
        path_to_videos=os.path.join(path_to_working_dir,k)
        print(path_to_videos)
        # get the number of videos within the sign folder 
        vids_num = len(os.listdir(path_to_videos))
        print(vids_num) 
        # initialize the array of the data 
        current_data=np.zeros((vids_num,frames_num,126))
        for i in range (vids_num): # loop through 30 video 
            path_to_current_sign= os.path.join(path_to_videos,f"output{i}.avi")
            
            # min_detection_confidence:The minimum confidence score for the hand detection to be 
            #considered successful in palm detection model.
            
            # min_tracking_confidence : The minimum confidence score for the hand tracking to be considered successful.
            #This is the bounding box IoU threshold between hands in the current frame and the last frame.
            #In Video mode and Stream mode of Hand Landmarker, if the tracking fails, Hand Landmarker 
            #triggers hand detection. Otherwise, it skips the hand detection.
            
            # start using the holistic model 
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :
                # check that the current sign folder exists 
                if os.path.exists(path_to_current_sign):

                    index = 0 
                    cap = cv2.VideoCapture(path_to_current_sign);  
                    length= int (cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    rest = length
                    start_frame =False 
                    # check that the video has the min number of frames required for extracting 
                    # meaningful information from it 
                    if(length>frames_num):
                        # open the vid 
                        while (cap.isOpened()):
                            ret , frame = cap.read()
                            # check if the cap return frame 
                            # and check if you have taken the required frames or not 
                            if ret == True and index<frames_num:  # keeping only the first 40 frame 
                                # detection  
                                image, results = mediapipe_detection(frame, holistic)
                                # extrating the keypoints 
                                keypoints = extract_keypoints(results,temp_nose_x,temp_nose_y)
                                
                                # start_frame will indicate if the model has detecetd a frame that 
                                # has some meaningful info . 
                                # meaningful means that not all value are zero ,as in the beginning
                                # of the video the singer starts by his hand not in the scope of the 
                                # camera , so this frames are not useful , all their values will be zero 
                                
                                # while the rest of the frames is more than the required number 
                                # and not strat frame 
                                if not(start_frame) and rest>frames_num:
                                    # if all ponits are zero and you didn't get any points before 
                                    # skip that frame as the number of rest frames is more than 
                                    # the required ones 
                                    if (np.all(keypoints==0)):
                                        rest-=1 
                                        continue
                                    else :
                                        start_frame=True

                                current_data[i][index]=keypoints

                                index+=1 
                            else :
                                print(f"this is video {i+1}")
                                break 

                        cap.release()
                        cv2.destroyAllWindows()

#         print("I am here will write to file ")
#         print(current_data[i])
      
        write_path = os.path.join(path_to_working_dir,k)
        print(write_path)
        if os.path.exists(write_path):
            print("yes")
            # save the data 
            np.save(os.path.join(write_path,"no_aug"),current_data)
        else : 
            pass 
    
    
    
    
    
    
    

                                

In [ ]:

def collect_data(all_vids,sign,path_to_working_dir,additional=False):
    """
    collect all the data of the one sign

    Args:
    all_Vids (list)           : List of all signs folder e.g.["Hello","Live","Love"] etc  
    path_to_working_dir (str) : Path to the directory of the vids 
    additional(boolean)       : boolean to indicate if this data is original or additional 
       
    Returns:
    None : nothing returned just save the keypoints in the file system 
      

    Description:
    Collect the data for the training processes , concatenate all the data of the differnent signs
    """
    X_train_no_aug=np.load(os.path.join(path_to_working_dir,all_vids[0],"no_aug.npy"))
    vids_num = X_train_no_aug.shape[0] 
    Y_train_no_aug=np.array([all_vids[0]]*vids_num)
    print(vids_num)

    for i in range (1, len(all_vids)):
        X_train_no_aug_temp=np.load(os.path.join(path_to_working_dir,all_vids[i],"no_aug.npy"))
        vids_num =  X_train_no_aug_temp.shape[0]
        print(vids_num)
        X_train_no_aug = np.concatenate([X_train_no_aug,X_train_no_aug_temp],axis=0)
        lst=[]
        lst.append(all_vids[i])
        Y_train_no_aug=np.concatenate([Y_train_no_aug,lst*vids_num])

    if additional : 
        np.save(os.path.join(path_to_working_dir,f"Data_train_clown_40_no_aug_yes_no_{sign}_additional.npy"),X_train_no_aug)
        np.save(os.path.join(path_to_working_dir,f"Data_labels_clown_40_no_aug_yes_no_{sign}_additional.npy"),Y_train_no_aug)
    else :
        np.save(os.path.join(path_to_working_dir,f"Data_train_clown_40_no_aug_yes_no_{sign}.npy"),X_train_no_aug)
        np.save(os.path.join(path_to_working_dir,f"Data_labels_clown_40_no_aug_yes_no_{sign}.npy"),Y_train_no_aug)




    

In [ ]:


def create_model(signs):
    """
    Creating the model.

    Args:
    Sings (dictionanry ) : dictionary contains all the signs in the vocab 
       
    Returns:
    model : Sequentail model , Recurrent Neural network . 
      

    Description:
    Creating the model that is required before training . 
    """
    model=Sequential()
#     print("IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII",len(signs))
    model.add(LSTM(64,return_sequences=False ,activation='relu',input_shape=(40,126)))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(signs), activation='softmax'))
    return model




In [ ]:


def load_old_model(signs,sign_name):
    """
    Loading Old Models for Re-training or Testing

    Args:

    Signs (dictionary): A dictionary that contains all the signs in the vocabulary.
    sign_name (str): The name of the sign whose model will be loaded.
    Returns:

    model: Sequential model, Recurrent Neural Network.
    Description:
    This function loads a pre-saved model for retraining or using it in the detection of new videos. It takes the dictionary of signs and the name of the sign whose model needs to be loaded as input.
    """
    
    # creating the model
    model = create_model(signs)
    # load its weights 
    model.load_weights(os.path.join(os.getcwd(),"models",f"{sign_name}_model.h5"))
    return model 

In [ ]:

class CustomCallback(keras.callbacks.Callback):
    """
        
    """
    
    
    def __init__(self,model_path="Iam_model.h5"):
        """
        Constructor of the Callback Class for Initializing Accuracies

        Args:

        model_path (str): The path at which the model will be saved.
        Returns:

        None
        Description:
        This constructor initializes the Callback class and is used for initializing accuracies. It takes the model path as input and does not return anything.
        """
        self.max_val_acc=0
        self.max_tra_acc=0
        self.model_path=model_path

    def on_epoch_end(self, epoch, logs=None):
        """
        Performing Actions at the End of Each Epoch

        Description:
        This function is called at the end of each epoch to perform certain actions. 
        Specifically, it saves the model if the accuracy achieved in the last epoch is 
        better than the accuracy of the current saved model.
        """
        tra_acc= logs.get("accuracy")
        if(tra_acc>0.98):
            self.model.stop_training=True 
            
        
        if  tra_acc > self.max_tra_acc :
            self.tra_acc=tra_acc
            self.model.save(self.model_path)
            

In [ ]:
def handle_data(X_train, Y_train,alphabets): 
    """
    Performing Checks on the Training Data and Label Conversion

    Args:

    X_train (numpy.ndarray): Numpy array containing the training data.
    Y_train (numpy.ndarray): Numpy array containing the labels of the training data.
    Returns:

    None
    Description:
    
    This function performs checks on the training data to ensure its validity for training. 
    First, it converts the string labels into their corresponding numerical labels for training purposes.
    Then, it performs two checks. The first check ensures that the conversion to one-hot vectors
    has been done correctly, meaning there are no labels without a logical one-hot vector representation.
    The second check verifies if there are any None values in the training data. 
    If any None values are found, the program halts.
    """
    
    Y_train_temp=np.zeros(Y_train.shape,dtype=np.int16)
    for i in range(Y_train.shape[0]):
        Y_train_temp[i]=alphabets[Y_train[i]]

    Y_train=Y_train_temp

    del Y_train_temp
    train_shuffler= np.random.permutation(X_train.shape[0])
    X_train=X_train[train_shuffler]
    Y_train=Y_train[train_shuffler]

    Y_train=tf.one_hot(Y_train,depth=len(alphabets))
    for item in Y_train:
        if( not np.any(item) and np.sum(item)!=1):
            assert("EROOR ")

    for i in range(X_train.shape[0]):
        for j in range(30):
            if(X_train[i][j][X_train[i][j]==None].shape[0] > 0 ):
                assert("Error")
    return X_train,Y_train 
    

In [ ]:
def retrain_model(sign,X_train,Y_train):
    """
    Retraining the Model for a Specific Sign on New Data

    Args:

    sign (str): The name of the model that will be trained on the new data.
    X_train (numpy.ndarray): Numpy array containing the new data that the model will be retrained on.
    Y_train (numpy.ndarray): Numpy array containing the labels of the new data.
    Returns:

    None: This function only saves the model.
    Description:
    This function is responsible for retraining old models on new data, specifically the newly added signs to the system. Since the old models have not seen the newly added data, all the old models will be retrained using samples from the newly added sign.
    """
    alphabets={sign:1,f"not_{sign}":0}
    X_train,Y_train=handle_data(X_train,Y_train,alphabets)
    model = load_old_model(alphabets,sign)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    callback = CustomCallback(os.path.join(os.getcwd(),"models",f"{sign}_model.h5"))
    print(X_train.shape)
    print(Y_train.shape)
    model.fit(X_train, Y_train,batch_size=32, epochs=800,verbose=1,callbacks=[callback])
    model.save(os.path.join(os.getcwd(),"models",f"{sign}_model.h5"))
    
    

In [ ]:
def train_model(sign,X_train,Y_train):
    """
    Training and Saving the Model for a Specific Sign

    Args:

    sign (str): The name of the sign for which the model will be dedicated.
    X_train (numpy.ndarray): Numpy array containing the training data to train the model.
    Y_train (numpy.ndarray): Numpy array containing the labels of the training data.
    Returns:

    None: This function only saves the model.
    Description:
    This function is responsible for training the models for a specific sign.
    These models will be used to check the detection of the sign, acting as a checker for 
    its detection. The function takes the sign name, training data, and labels as input, 
    and saves the trained model for the sign.

    """
    alphabets={sign:1,f"not_{sign}":0}
    Y_train_temp=np.zeros(Y_train.shape,dtype=np.int16)

    X_train,Y_train=handle_data(X_train,Y_train,alphabets)
    model=create_model(alphabets)      
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    callback = CustomCallback(os.path.join(os.getcwd(),"models",f"{sign}_model.h5"))
    model.fit(X_train, Y_train,batch_size=32, epochs=1700,verbose=1,callbacks=[callback])
    model.save(os.path.join(os.getcwd(),"models",f"{sign}_model.h5"))

In [ ]:
def create_not_folder(not_vids,path_to_not_data,sign,index,path_to_vid):
    """
    Creating the Folder for Samples of Signs that are not Equal to the Current Sign

    Args:

    not_vids (list): A list containing all the signs that are not equal to the current sign.
    path_to_not_data (str): The directory that contains the folder of videos for signs that are not the current sign.
    sign (str): The name of the current sign.
    path_to_vid (str): The path to the folder of videos for the current sign.
    Description:
    This function creates the "not_sign_folder" which contains samples from all the signs that are not equal to the current sign. This folder is used to train the model to distinguish between the current sign and all the other signs. It takes the list of not equal signs, the path to the folder of videos for signs that are not the current sign, the name of the current sign, and the path to the folder of videos for the current sign.
    """
    previous_rand=-1 
    vids_num = len(os.listdir(os.path.join(path_to_not_data,sign)))
    shutil.copyfile(os.path.join(path_to_not_data,sign,f"output0.avi"),
                    os.path.join(not_sign_path,f"output{index}.avi"))
    index+=1 
    shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{1}.avi"),
                    os.path.join(not_sign_path,f"output{index}.avi"))
    index+=1 

    new_rand= np.random.randint(11,vids_num)
    while (new_rand==previous_rand):
        new_rand= np.random.randint(11,vids_num)

    previous_rand=new_rand

    shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{new_rand}.avi"),
                    os.path.join(not_sign_path,f"output{index}.avi"))
    index+=1 

    new_rand= np.random.randint(11,vids_num)
    while (new_rand==previous_rand):
        new_rand= np.random.randint(11,vids_num)

    previous_rand = new_rand 

    shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{new_rand}.avi"),
                    os.path.join(not_sign_path,f"output{index}.avi"))
    index+=1 

    num_of_vid_in_sign=len(os.listdir(path_to_vid))

    num_of_vid_in_not_sign=len(os.listdir(not_sign_path))

    while(num_of_vid_in_not_sign > num_of_vid_in_sign):
        new_rand=np.random.randint(0,num_of_vid_in_sign)
        if(new_rand>=num_of_vid_in_sign-1):
            continue 
        shutil.copyfile(os.path.join(path_to_vid,f"output{new_rand}.avi")
                        ,os.path.join(path_to_vid,f"output{num_of_vid_in_sign}.avi"))
        num_of_vid_in_sign=len(os.listdir(path_to_vid))
    
    

In [ ]:
def save_training_data(sign_name,new_added_dir):
    """
    Saving the Training Data

    Args:

    sign_name (str): The name of the sign.
    new_added_dir (str): The path to the working directory used to save the data.
    Returns:

    None: This function only saves the data.
    Description:
    This function is responsible for saving the training data of the created signs. It takes the sign name and the path to the working directory as input and saves the data in the specified directory.
    """
    if(os.path.exists(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))) : 
        os.remove(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))
    shutil.move(os.path.join(os.getcwd(),new_added_dir,f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"),
                os.path.join(os.getcwd(),"Data_points"))

    if(os.path.exists(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))) : 
        os.remove(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))

    shutil.move(os.path.join(os.getcwd(),new_added_dir,f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"),
                os.path.join(os.getcwd(),"Data_points"))  

In [ ]:
def get_X_Y_train(new_added_dir,sign_name):
    """
    Collecting and Returning Training Data

    Args:

    new_added_dir (str): The path to the working directory used to save the data.
    sign_name (str): The name of the sign.
    Returns:

    X_train (numpy.ndarray): The collected data, ready for training.
    Y_train (numpy.ndarray): The labels of the collected data.
    Description:
    This function is responsible for collecting the data and returning it in the form of X_train and Y_train. It takes the path to the working directory and the sign name as input. The function collects the data from the specified directory and returns the collected data along with its corresponding labels.
    """
    os.remove(os.path.join(new_added_dir,f"{sign_name}","no_aug.npy"))
    shutil.move(os.path.join(os.getcwd(),new_added_dir,sign_name),
                os.path.join(os.getcwd(),"data_videos",sign_name))
    shutil.rmtree(os.path.join(os.getcwd(),new_added_dir,f"not_{sign_name}"))
    X_train=np.load(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))
    Y_train=np.load(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))
    return X_train, Y_train 

In [ ]:
# choice = int (input("Choose to record new videos=>0 or you have data in the new added vidoes folder ==>1"))

path_to_videos=os.path.join(os.getcwd(),"data_videos/")
list_of_vids = os.listdir(path_to_videos)

for sign_video_index in list_of_vids:
    
    sign_name=sign_video_index
    shutil.move(os.path.join(path_to_videos,sign_name),
                os.path.join(os.getcwd(),"New_added_Videos"))
    new_added_dir = os.path.join(os.getcwd(),"New_added_Videos")
    while True:
        if(os.path.exists(os.path.join( new_added_dir,sign_name))):
            path_to_vid = os.path.join(os.getcwd(),"New_added_Videos",sign_name)
            break 
        else : 
            sign_name=(input("Please enter valid name of the sign"))
            path_to_vid = os.path.join(os.getcwd(),"New_added_Videos",sign_name)
    path_to_not_data=os.path.join(os.getcwd(),"data_videos") 
    not_vids = os.listdir(path_to_not_data)
    print(not_vids)
    not_sign_path=os.path.join(new_added_dir,f"not_{sign_name}")
    os.mkdir(not_sign_path)
    index = 0
    previous_rand=-1 
    for sign in not_vids: 

        create_not_folder(not_vids,path_to_not_data,sign,index,path_to_vid)


    all_vids=[f"{sign_name}",f"not_{sign_name}"]

    generate_training_data(all_vids, new_added_dir)
    collect_data(all_vids,sign_name,new_added_dir)

    save_training_data(sign_name,new_added_dir)  

    X_train,Y_train= get_X_Y_train(new_added_dir,sign_name)
    train_model(sign_name,X_train,Y_train)

## retrain the previous trained models to adapt with the new signs 

In [ ]:
def retrain_more_signs(new_list):
    """
    retrain more signs and get more models for them 
    
    Args: 
    new_list(list): the list of signs that will be added 
    
    Returns : 
    None 
    
    Description : 
    this function is used if you want to add new signs , it will retrain all the previous model , 
    so it's suggest to use this funciion with big new list not with few ones , becuase all the models 
    will be trained . 
    the function sample four videos from all new list and put them in the not_sign for the model which 
    is re-trained and it will loop over all models , to let the old models to learn that the new signs 
    don't belong to them  , to be able to differintiate between the signs 
    """ 
    except_list=[]
    entire_data_path = os.path.join(os.getcwd(),"data_videos/")
    entire_list=os.listdir(entire_data_path)
    old_list=[]
    for i in entire_list :
        if i not in new_list:
            old_list.append(i)

    print(old_list)

    new_data_path  = os.path.join(os.getcwd(),"New_added_Videos/")
    for i in old_list:
        if(i in except_list ) : continue 
        not_sign_path = os.path.join(new_data_path,f"not_{i}")
        os.mkdir(not_sign_path) ; 
        index = 0  
        for j in new_list:
            path_to_curr_sign=os.path.join(entire_data_path,j)
            shutil.copyfile(os.path.join(path_to_curr_sign,f"output{0}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 
            shutil.copyfile(os.path.join(path_to_curr_sign,f"output{1}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 
            curr_sign_num_vids = len(os.listdir(path_to_curr_sign))
            rand_int = np.random.randint(2,curr_sign_num_vids)

            shutil.copyfile(os.path.join(path_to_curr_sign,f"output{rand_int}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 

            rand_int = np.random.randint(2,curr_sign_num_vids)

            shutil.copyfile(os.path.join(path_to_curr_sign,f"output{rand_int}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 
            print(j)



        num_of_not_sign = len (os.listdir(not_sign_path))
        os.mkdir(os.path.join(new_data_path,i))
        index = 0 
        for k in range(num_of_not_sign): 
            num_of_sign= len(os.listdir(os.path.join(entire_data_path,i)))
            randint = np.random.randint(0 ,num_of_sign-1)
            shutil.copyfile(os.path.join(entire_data_path,i,f"output{randint}.avi"),
                           os.path.join(new_data_path,i,f"output{index}.avi"))
            index+=1 

        all_vids=[f"{i}",f"not_{i}"]

        generate_training_data(all_vids, new_data_path)
        collect_data(all_vids,i,new_data_path,True)

        path_to_data_points = os.path.join(os.getcwd(),"Data_points")

        old_X_train= np.load(os.path.join(path_to_data_points,
                                          f"Data_train_clown_40_no_aug_yes_no_{i}.npy"))
        old_Y_train= np.load(os.path.join(path_to_data_points,
                                          f"Data_labels_clown_40_no_aug_yes_no_{i}.npy"))

        X_temp_train=np.load(os.path.join(new_data_path,
                                         f"Data_train_clown_40_no_aug_yes_no_{i}_additional.npy"))
        Y_temp_train= np.load(os.path.join(new_data_path,
                                          f"Data_labels_clown_40_no_aug_yes_no_{i}_additional.npy"))

        X_train=np.concatenate([old_X_train,X_temp_train])
        Y_train=np.concatenate([old_Y_train,Y_temp_train])
        retrain_model(i,X_train,Y_train)

        if(os.path.exists(os.path.join(path_to_data_points,f"Data_train_clown_40_no_aug_yes_no_{i}.npy"))):
            os.remove(os.path.join(path_to_data_points,f"Data_train_clown_40_no_aug_yes_no_{i}.npy"))
            np.save(os.path.join(path_to_data_points,f"Data_train_clown_40_no_aug_yes_no_{i}.npy"),X_train)

        if(os.path.exists(os.path.join(path_to_data_points,f"Data_labels_clown_40_no_aug_yes_no_{i}.npy"))):
            os.remove(os.path.join(path_to_data_points,f"Data_labels_clown_40_no_aug_yes_no_{i}.npy"))
            np.save(os.path.join(path_to_data_points,f"Data_labels_clown_40_no_aug_yes_no_{i}.npy"),Y_train)

        shutil.rmtree(os.path.join(new_data_path,i))
        shutil.rmtree(os.path.join(new_data_path,f"not_{i}"))
        os.remove(os.path.join(new_data_path,
                               f"Data_labels_clown_40_no_aug_yes_no_{i}_additional.npy"))

        os.remove(os.path.join(new_data_path,
                               f"Data_train_clown_40_no_aug_yes_no_{i}_additional.npy"))








    



In [ ]:
print(Y_train)

In [ ]:



################################33back up 


# choice = int (input("Choose to record new videos=>0 or you have data in the new added vidoes folder ==>1"))
choice = 1 ;
path_to_videos=os.path.join(os.getcwd(),"data_videos/")
list_of_vids = os.listdir(path_to_videos)

for sign_video_index in list_of_vids:

    if (choice ==1 ): 
        print("Vidoes")
        sign_name=sign_video_index
        shutil.move(os.path.join(path_to_videos,sign_name),
                    os.path.join(os.getcwd(),"New_added_Videos"))
        new_added_dir = os.path.join(os.getcwd(),"New_added_Videos")
        while True:
            if(os.path.exists(os.path.join( new_added_dir,sign_name))):
                path_to_vid = os.path.join(os.getcwd(),"New_added_Videos",sign_name)
                break 
            else : 
                sign_name=(input("Please enter valid name of the sign"))
                path_to_vid = os.path.join(os.getcwd(),"New_added_Videos",sign_name)
        # first secinario , there is no previous data , and no retrain , just new sign 
        path_to_not_data=os.path.join(os.getcwd(),"data_videos") 
        not_vids = os.listdir(path_to_not_data)
        print(not_vids)
        not_sign_path=os.path.join(new_added_dir,f"not_{sign_name}")
        os.mkdir(not_sign_path)
        index = 0

        previous_rand=-1 
        for sign in not_vids: 

            vids_num = len(os.listdir(os.path.join(path_to_not_data,sign)))
    #         print(vids_num)
            shutil.copyfile(os.path.join(path_to_not_data,sign,f"output0.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 
            shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{1}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 

            new_rand= np.random.randint(11,vids_num)
            while (new_rand==previous_rand):
                new_rand= np.random.randint(11,vids_num)

            previous_rand=new_rand

            shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{new_rand}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 

            new_rand= np.random.randint(11,vids_num)
            while (new_rand==previous_rand):
                new_rand= np.random.randint(11,vids_num)

            previous_rand = new_rand 

            shutil.copyfile(os.path.join(path_to_not_data,sign,f"output{new_rand}.avi"),
                            os.path.join(not_sign_path,f"output{index}.avi"))
            index+=1 

            num_of_vid_in_sign=len(os.listdir(path_to_vid))

            num_of_vid_in_not_sign=len(os.listdir(not_sign_path))
    #         print(num_of_vid_in_sign)
    #         print(num_of_vid_in_not_sign)
            while(num_of_vid_in_not_sign > num_of_vid_in_sign):
                new_rand=np.random.randint(0,num_of_vid_in_sign)
                if(new_rand>=num_of_vid_in_sign-1):
                    continue 
                shutil.copyfile(os.path.join(path_to_vid,f"output{new_rand}.avi")
                                ,os.path.join(path_to_vid,f"output{num_of_vid_in_sign}.avi"))
                num_of_vid_in_sign=len(os.listdir(path_to_vid))
                print("Peop",num_of_vid_in_sign,num_of_vid_in_not_sign)



        all_vids=[f"{sign_name}",f"not_{sign_name}"]

        generate_training_data(all_vids, new_added_dir)
        collect_data(all_vids,sign_name,new_added_dir)
        print(new_added_dir)
        if(os.path.exists(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))) : 
            os.remove(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))
        shutil.move(os.path.join(os.getcwd(),new_added_dir,f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"),
                   os.path.join(os.getcwd(),"Data_points"))

        if(os.path.exists(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))) : 
            os.remove(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))

        shutil.move(os.path.join(os.getcwd(),new_added_dir,f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"),
                    os.path.join(os.getcwd(),"Data_points"))  

    #     for vid in all_vids:
    #         os.remove(os.path.join(new_added_dir,f"{sign_name}","no_aug.npy"))
        os.remove(os.path.join(new_added_dir,f"{sign_name}","no_aug.npy"))
        shutil.move(os.path.join(os.getcwd(),new_added_dir,sign_name),
                    os.path.join(os.getcwd(),"data_videos",sign_name))
        shutil.rmtree(os.path.join(os.getcwd(),new_added_dir,f"not_{sign_name}"))
        X_train=np.load(os.path.join(os.getcwd(),"Data_points",f"Data_train_clown_40_no_aug_yes_no_{sign_name}.npy"))
        Y_train=np.load(os.path.join(os.getcwd(),"Data_points",f"Data_labels_clown_40_no_aug_yes_no_{sign_name}.npy"))
        train_model(sign_name,X_train,Y_train)

    else : 
        print("recording")